## <center> Exploring CMIP6 Data </center>
Started 10/28/24

Discription of the nomenclature used for the runs is provided at this link https://ukesm.ac.uk/cmip6/variant-id/

In [1]:
import xarray as xr
import numpy as np
import calendar
import pandas as pd
import datetime
import os
import cftime

### Things to note:
- Right now we are only using one ensamble member from each simulation (generally the first one)
    - except for BCC-CSM2-MR where we use the second ensamble member since the first one is missing
- KACE-1-0-G the simulation code for tas and huss do not match d20200316 and d20190911 respectivly - need to investigate if these are even coming from the same simulation in this case feels like a mistake - simmilarly BCC-CSM2-MR does not have matching codes either - it will make retrieving additional variables more challenging as you'll have to add in conditions each time this occurs
- On further research this weird number is actually the version but you can always just link to latest - that should be the default unless there is only one directory there in which case you need to do the only one in it? - a little confused there
- Quite nicely all the variables seem to be labled similarly
- The time dimensions are not always the same in length - potentially some don't include leap years or don't include 31sts?
- For now I'm using this NASA IMERG_land_sea_mask dataset however it is just purely what % of the earth has land and what has water - so it includes lakes - one potential issue with this is that sometimes it ends up including some of the land as ocean which is more problematic in higher latitudes - ask mike about it but it might be an issue for later - alternativly you could check its accuracy against the ones that are included (which is still like 8)
###### On Forcings
- Generally we use the forcing #1 (ie the input4MIPs version v6.1.1: no ozone remapping) however for 4 models (CNRM-ESM2-1, CNRM-CM6-1, MIROC-ES2L and UKESM1-0-LL) we use forcing 2 (Input4MIPs version v6.2.0; no ozone remapping) and for one (HadGEM3-GC31-LL) we use forcing 3 (Input4MIPs version v6.2.0; with ozone remapping)
- On the UKESM website they say that outside of the stratosphere they dont expext differences in forcing to have significant impact on model behaviour and recommend that users who are not looking at stratospheric chemistry use all three members in the main historical ensemble https://ukesm.ac.uk/cmip6/variant-id/ 

## Finding land fraction Data


In [2]:
def getVariable(variable, modelName, simulations):

    modelNames = pd.read_excel('/home/users/chingosa/CMIP6/CMIP6Models.xlsx')
    i = modelNames.index[modelNames['ModelName'] == modelName].tolist()[0]
    org = modelNames.ModelInstitution[i]
    model = modelNames.ModelName[i]
    fCode = modelNames.f[i]
    grid = modelNames.grid[i]
    ensambleMember = 1
    period = 'day'
    if variable == 'sftof': period = 'Ofx'

    #File Path for historical and ssp all daily variable
    if simulations == 'ssp245':
        folder_path = f'/badc/cmip6/data/CMIP6/ScenarioMIP/{org}/{model}/ssp245/r{ensambleMember}i1p1f{fCode}/{period}/{variable}/{grid}/files/'     
    elif simulations =='historical':
        
        if model in ['BCC-CSM2-MR'] : ensambleMember += 1 # forwhatever reason there is no 1st ensamble member for this one
        folder_path = f'/badc/cmip6/data/CMIP6/CMIP/{org}/{model}/historical/r{ensambleMember}i1p1f{fCode}/{period}/{variable}/{grid}/files/'
    
    if len(os.listdir(folder_path)) == 1:
        folder_path = os.path.join(folder_path, os.listdir(folder_path)[0])
    else:
        folder_path = os.path.join(folder_path, 'latest')
            
    # List and print all file names in the specified folder
    file_paths = [
    os.path.join(folder_path, file_name) 
    for file_name in os.listdir(folder_path) 
    if os.path.isfile(os.path.join(folder_path, file_name))]
        
    return file_paths

## This is how we will do land sea for now

In [50]:
# Its not perfect but the data set is the issue...
def addLandMask(ds):
    '''
    Takes in a dataset finds the variable that his lat lon coords and makes a 2D data array with the coursened version of the land mask
    Adds that back into the ds
    '''
    landMask = xr.open_dataarray('CMIP6/IMERG_land_sea_mask.nc')
    vars = list(ds.keys())
    
    for i in np.arange(len(vars)):
        if set(['lat', 'lon']).issubset(list(ds[vars[i]].coords)):
            chosenVar = list(ds.keys())[i]
    
    da = ds[chosenVar].isel(time=0) # this line would be an issue if there isn't time variable
    coarsened_da = landMask.interp_like(da)
    coarsened_da = (coarsened_da/100)
    coarsened_da = xr.apply_ufunc(np.round, coarsened_da)
    ds['landseamask'] = coarsened_da
    return ds
    
tas = xr.open_mfdataset(getVariable('tas', 'NorESM2-LM', 'ssp245'))
tas = addLandMask(tas)


## And this is how we will slice/dice

In [28]:
def regionTimeCut(ds, period):
    if period == 'historical':
        start_date = cftime.DatetimeNoLeap(1980, 1, 1)
        end_date = cftime.DatetimeNoLeap(2000, 12, 31)
    elif period == 'ssp245':
        start_date = cftime.DatetimeNoLeap(2080, 1, 1)
        end_date = cftime.DatetimeNoLeap(2100, 12, 31)

    if ds.time.dtype == '<M8[ns]':
        start_date, end_date  = np.datetime64(start_date), np.datetime64(end_date)
    elif type(ds.time.values[0]) == cftime._cftime.Datetime360Day:
        if period == 'historical':
            start_date = cftime.Datetime360Day(1980, 1, 1)
            end_date = cftime.Datetime360Day(2000, 12, 30)
        elif period == 'ssp245':
            start_date = cftime.Datetime360Day(2080, 1, 1)
            end_date = cftime.Datetime360Day(2100, 12, 30)
            
    tropical_region = ds.sel(lat=slice(-20, 20))
    tropical_region = tropical_region.sel(time=slice(start_date, end_date))
    return tropical_region

modelNames = pd.read_excel('/home/users/chingosa/CMIP6/CMIP6Models.xlsx')
ds = xr.open_mfdataset(getVariable('tas', 'HadGEM3-GC31-LL', 'ssp245'))
ds = regionTimeCut(ds, 'ssp245')
ds

<xarray.Dataset> Size: 213MB
Dimensions:    (time: 7559, bnds: 2, lat: 32, lon: 192)
Coordinates:
  * time       (time) object 60kB 2080-01-01 12:00:00 ... 2100-12-29 12:00:00
  * lat        (lat) float64 256B -19.38 -18.12 -16.88 ... 16.88 18.12 19.38
  * lon        (lon) float64 2kB 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
    height     float64 8B 1.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 121kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 4MB dask.array<chunksize=(7559, 32, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 23MB dask.array<chunksize=(7559, 192, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 186MB dask.array<chunksize=(1, 32, 192), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   59400.0
    branch_time_in_parent:  59400.0
    creation_date:          2019-09-06T15:32:31Z
    ...                     ...
    tracking_id:            hdl:21.14100/16025ab0-7096-4199-b180-d3f2b6b9c0a9
    variable_id:            tas
    variable_name:          tas
    variant_label:          r1i1p1f3
    license:                CMIP6 model data produced by the Met Office Hadle...
    cmor_version:           3.4.0